In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%autosave 60

Autosaving every 60 seconds


# DSCI 235 Final Project
### By: Blake Davis and Elisabeth Ball

Data Source 1: https://www.kaggle.com/datasets/aliaamiri/2020-summer-olympics-dataset?select=2020_Olympics_Dataset.csv <br>
Data Source 2: https://www.kaggle.com/datasets/rio2016/olympic-games <br>
GitHub: https://github.com/bdavis12302/2020-2016-Summer-Olympic-Data <br>

## Project Description

For this project, we are analyzing data from the Tokyo 2020 Olympics and the Rio 2016 Olympics. The Olympic Games are a fascinating event, bringing the world together to appreciate excellence in athletics. The numerous different events, countries, and people competing in the Olympic Games give ample material to analyze. Finding patterns and interesting information in this Olympic data will likely change the way we see the games in the future and make us think about the nature of sports in a new way. This project will also be challenging and help us develop our teamwork, communication, and data wrangling skills. <br>

We are using 2 .csv files for our project. We have a main data set that is for the 2020 Summer Olympics in Tokyo. It includes data sorted by athlete name, with information on each athlete including age, country, sport, event, rank, and medal earned. The data set has multiple entries for each athlete as many athletes compete in multiple events. We have not decided how we are going to sort/filter the data for each individual athlete as they are each in the data set multiple times. We will use a 2016 Olympics in Rio De Janeiro data set to be able to determine if there were any repeat medalists and if there were more female or male medalists in 2016 vs. 2020. <br>

<hr>

We want to answer a few questions about the athletes that chose to compete in the 2016 and 2020 Summer Olympics:
1. What is the average age of an Olympic athlete in 2016? 2020? Compare.
   - Are there events that tend to have older compeditors? Meaning, is the mean age of athletes competing an a specific event greater than 30?<br>
6. Who are the 5 oldest athletes and who are the 5 youngest athletes in 2020?
   - What country are they from? <br>
   - What sport do they compete in? What is the age distribution of their sport in 2020? <br>
   - Did any of these athletes compete in 2016?
   - Do any of them compete in the events that tend to have older compeditors?
3. What are the top medaling countries in 2016? 2020? Compare. <br>
4. Does the United States have more female or male medalists in 2020? <br>
5. Which Olympic athlete has the most medals in 2016? 2020?
   - Is it the same person each Olympics? Same event? <br>

In [14]:
olympics2020 = pd.read_csv('https://raw.githubusercontent.com/bdavis12302/2020-2016-Summer-Olympic-Data/main/SummerOlympicDatasets/2020_Olympics_Dataset.csv', encoding='latin-1')
olympics2016 = pd.read_csv('https://raw.githubusercontent.com/bdavis12302/2020-2016-Summer-Olympic-Data/main/SummerOlympicDatasets/2016_athletes.csv', encoding='latin-1')
olympics2016.head()

,id,name,nationality,sex,dob,height,weight,sport,gold,silver,bronze
0,736041664,A Jesus Garcia,ESP,male,10/17/69,1.72,64.0,athletics,0,0,0
1,532037425,A Lam Shin,KOR,female,9/23/86,1.68,56.0,fencing,0,0,0
2,435962603,Aaron Brown,CAN,male,5/27/92,1.98,79.0,athletics,0,0,1
3,521041435,Aaron Cook,MDA,male,1/2/91,1.83,80.0,taekwondo,0,0,0
4,33922579,Aaron Gate,NZL,male,11/26/90,1.81,71.0,cycling,0,0,0
